In [265]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('database.sqlite')
cur = conn.cursor()
from pandasql import sqldf
import matplotlib.pyplot as plt
%matplotlib inline
pysqldf = lambda q: sqldf(q, globals())
import numpy as np
#table names - matches / teams / teams_in_matches / unique_teams
# table columns TeamName, Unique_Team_ID, Match_ID, Season, Date, HomeTeam, AwayTeam, FTHG, FTAG, FTR

In [76]:
#selecting the matches table
cur.execute("""SELECT * FROM matches""")

df_matches = pd.DataFrame(cur.fetchall())
df_matches.columns = [x[0] for x in cur.description]
df_matches.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1,D2,2009,2010-04-04,Oberhausen,Kaiserslautern,2,1,H
1,2,D2,2009,2009-11-01,Munich 1860,Kaiserslautern,0,1,A
2,3,D2,2009,2009-10-04,Frankfurt FSV,Kaiserslautern,1,1,D
3,4,D2,2009,2010-02-21,Frankfurt FSV,Karlsruhe,2,1,H
4,5,D2,2009,2009-12-06,Ahlen,Karlsruhe,1,3,A


In [77]:
# selecting the matches table with matches in season 2011
cur.execute("""SELECT * FROM matches
            WHERE season == 2011
                        
            """)

df_matches = pd.DataFrame(cur.fetchall())
df_matches.columns = [x[0] for x in cur.description]
df_matches.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D


In [78]:
# creating a new column in the df_matches dataframe which shows the winner of the match
for i, row in df_matches.iterrows():
    if row.FTR == "H":
        df_matches.loc[i,'Winner'] = df_matches.loc[i,"HomeTeam"]
    elif row.FTR == "A":
        df_matches.loc[i,'Winner'] = df_matches.loc[i,"AwayTeam"]
    else:
        df_matches.loc[i,'Winner'] = "Draw"

In [79]:
df_matches.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Winner
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,Bayern Munich
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,Bayern Munich
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,Bayern Munich
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H,Mainz
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D,Draw


In [330]:
# creating a new data frame 'df' which merges the tables unique_teams, matches and unique_team_id
cur.execute("""SELECT * FROM unique_teams
            JOIN teams_in_matches
            USING (unique_team_id)
            JOIN matches
            USING (match_id)
            WHERE season == 2011
                        
            """)

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,TeamName,Unique_Team_ID,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,Bayern Munich,1,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,Nurnberg,22,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
2,Bayern Munich,1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
3,Stuttgart,17,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
4,Bayern Munich,1,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A


In [331]:
# creating a new column in the dataframe 'df' using iterrows, this loop will create a new column
# 'outcome' assigning a W, L or D based on the match result
for i, row in df.iterrows():
    if row.TeamName == row.HomeTeam:
        if row.FTHG > row.FTAG:
            df.loc[i,'Outcome'] = 'W'
        elif row.FTHG < row.FTAG:
            df.loc[i,'Outcome'] = 'L'
        elif row.FTHG == row.FTAG:
            df.loc[i,'Outcome'] = 'D'
    elif row.TeamName == row.AwayTeam:
        if row.FTAG > row.FTHG:
            df.loc[i,'Outcome'] = 'W'
        elif row.FTAG < row.FTHG:
            df.loc[i,'Outcome'] = 'L'
        elif row.FTAG == row.FTHG:
            df.loc[i,'Outcome'] = 'D'
        
    

In [332]:
df.head()

,TeamName,Unique_Team_ID,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Outcome
0,Bayern Munich,1,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,W
1,Nurnberg,22,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,L
2,Bayern Munich,1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,W
3,Stuttgart,17,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,L
4,Bayern Munich,1,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,W


In [335]:
# this loop will create a new column on the data frame using iterrows.
#it compares the FTHG and FTAG columns, and based on this creates a new column total goal,
#which shows the total goals scored by the 'TeamName'
for i, row in df.iterrows():
    if row.TeamName == row.HomeTeam:
        df.loc[i,'Total_Goals_by_Team_Name'] = df.loc[i,'FTHG']
    else:
        df.loc[i,'Total_Goals_by_Team_Name'] = df.loc[i,'FTAG']

In [336]:
df.head()

,TeamName,Unique_Team_ID,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Outcome,Total_Goals_by_Team_Name
0,Bayern Munich,1,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,W,1.0
1,Nurnberg,22,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,L,0.0
2,Bayern Munich,1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,W,2.0
3,Stuttgart,17,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,L,1.0
4,Bayern Munich,1,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,W,1.0


In [337]:
# using pysql package to filter the df dataframe by TeamName, and taking an aggregate of the outcome
#column using a filter L, to indicate the team losses.
p = """SELECT TeamName, COUNT (Outcome) AS TotalLoss FROM df
        WHERE outcome = 'L'
        GROUP BY TeamName
        """

total_loss = pysqldf(p)
total_loss.set_index('TeamName', inplace=True)
total_loss.head()



,TotalLoss
TeamName,
Aachen,15
Arsenal,10
Aston Villa,14
Augsburg,12
Bayern Munich,7


In [191]:
# using pysql package to filter the df dataframe by TeamName, and taking an aggregate of the outcome
#column using a filter W, to indicate the team wins.
q = """SELECT TeamName, COUNT (Outcome) AS TotalWins FROM df
        WHERE outcome = 'W'
        GROUP BY TeamName
        """


In [338]:
total_wins = pysqldf(q)
total_wins.set_index('TeamName', inplace=True)
total_wins.head()


,TotalWins
TeamName,
Aachen,6
Arsenal,21
Aston Villa,7
Augsburg,8
Bayern Munich,23


In [339]:
# using pysql package to filter the df dataframe by TeamName, and taking an aggregate of the goals scored

z = """SELECT TeamName, SUM(Total_Goals_by_team_name) AS TotalGoals
        FROM df
        GROUP BY TeamName
        """

In [340]:
total_goals = pysqldf(z)
total_goals.set_index('TeamName', inplace=True)
total_goals.head()

,TotalGoals
TeamName,
Aachen,30.0
Arsenal,74.0
Aston Villa,37.0
Augsburg,36.0
Bayern Munich,77.0


In [343]:
# using pysql package to filter the df dataframe by TeamName, and taking an aggregate of 
#total number of matches played by each team


y = """SELECT TeamName, COUNT(Match_ID) AS TotalMatches
        FROM df
        GROUP BY TeamName
        """

In [344]:
total_matches = pysqldf(y)
total_matches.set_index('TeamName', inplace=True)
total_matches.head()


,TotalMatches
TeamName,
Aachen,34
Arsenal,38
Aston Villa,38
Augsburg,34
Bayern Munich,34


In [345]:
#joining the 3 dataframes together
df_joined = total_matches.join(total_wins).join(total_loss).join(total_goals)


In [346]:
#resetting the index column
df_joined.reset_index(inplace=True)

In [347]:
df_joined.head()

,TeamName,TotalMatches,TotalWins,TotalLoss,TotalGoals
0,Aachen,34,6,15,30.0
1,Arsenal,38,21,10,74.0
2,Aston Villa,38,7,14,37.0
3,Augsburg,34,8,12,36.0
4,Bayern Munich,34,23,7,77.0


In [348]:
df.head()

,TeamName,Unique_Team_ID,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Outcome,Total_Goals_by_Team_Name
0,Bayern Munich,1,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,W,1.0
1,Nurnberg,22,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,L,0.0
2,Bayern Munich,1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,W,2.0
3,Stuttgart,17,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,L,1.0
4,Bayern Munich,1,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,W,1.0


In [351]:
df['Date'] = pd.to_datetime(df['Date'], format='%Y/%m/%d')

In [360]:
df['Date Unix'] = df['Date'].astype(np.int64) // 10**9

In [361]:
df.head()

,TeamName,Unique_Team_ID,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Outcome,Total_Goals_by_Team_Name,Date Unix
0,Bayern Munich,1,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,W,1.0,1333152000
1,Nurnberg,22,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,L,0.0,1333152000
2,Bayern Munich,1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,W,2.0,1323561600
3,Stuttgart,17,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,L,1.0,1323561600
4,Bayern Munich,1,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,W,1.0,1313193600


In [352]:
df['Date'][0]

Timestamp('2012-03-31 00:00:00')

In [356]:
type(df['Date'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [68]:
cur.execute("""SELECT * FROM unique_teams
            JOIN teams_in_matches
            USING (unique_team_id)
            JOIN matches
            USING (match_id)
            WHERE season == 2011
                        
            """)

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,TeamName,Unique_Team_ID,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,Bayern Munich,1,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,Nurnberg,22,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
2,Bayern Munich,1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
3,Stuttgart,17,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
4,Bayern Munich,1,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A


In [ ]:
for i, row in df.iterrows():
    if row.FTR == "H":
        df.loc[i,'Winner'] = df_matches.loc[i,"HomeTeam"]
    elif row.FTR == "A":
        df.loc[i,'Winner'] = df_matches.loc[i,"AwayTeam"]
    else:
        df.loc[i,'Winner'] = "D"

In [20]:
for i, row in df.iterrows():
    if row.FTR == "H":
        df.loc[i,'Winner'] = df.loc[i,"HomeTeam"]
    elif row.FTR == "A":
        df.loc[i,'Winner'] = df.loc[i,"AwayTeam"]
    else:
        df.loc[i,'Winner'] = "D"

In [25]:
for i, row in df.iterrows():
    if row.TeamName == row.HomeTeam:
        df.loc[i,'TeamNameGoals'] = df.loc[i,'FTHG']
    else:
        df.loc[i,'TeamNameGoals'] = df.loc[i,'FTAG']

In [26]:
df

,TeamName,Unique_Team_ID,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Winner,TeamNameGoals
0,Bayern Munich,1,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,Bayern Munich,1.0
1,Nurnberg,22,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,Bayern Munich,0.0
2,Bayern Munich,1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,Bayern Munich,2.0
3,Stuttgart,17,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,Bayern Munich,1.0
4,Bayern Munich,1,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,Bayern Munich,1.0
5,Wolfsburg,7,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,Bayern Munich,0.0
6,Bayern Munich,1,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H,Mainz,2.0
7,Mainz,11,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H,Mainz,3.0
8,Bayern Munich,1,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D,D,0.0
9,Freiburg,16,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D,D,0.0


In [6]:
for i, row in df.iterrows():
    if row.TeamName == row.AwayTeam:
        # Away
        if row.FTR == 'A':
            df.loc[i,'Result'] = "W"
        elif row.FTR == 'D':
            df.loc[i,'Result'] = "D"
        else:
            df.loc[i,'Result'] = "L"
    else:
        # Home
        if row.FTR == 'H':
            df.loc[i,'Result'] = "W"
        elif row.FTR == 'D':
            df.loc[i,'Result'] = "D"
        else:
            df.loc[i,'Result'] = "L"

In [7]:
df.head()

,TeamName,Unique_Team_ID,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Result
0,Bayern Munich,1,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,W
1,Nurnberg,22,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,L
2,Bayern Munich,1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,W
3,Stuttgart,17,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,L
4,Bayern Munich,1,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,W


In [63]:
df_matches['winner'] = df_matches['FTHG'] > df_matches['FTAG']

In [64]:
df_matches.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,winner,winner2
0,1,D2,2009,2010-04-04,Oberhausen,Kaiserslautern,2,1,H,True,False
1,2,D2,2009,2009-11-01,Munich 1860,Kaiserslautern,0,1,A,False,False
2,3,D2,2009,2009-10-04,Frankfurt FSV,Kaiserslautern,1,1,D,False,False
3,4,D2,2009,2010-02-21,Frankfurt FSV,Karlsruhe,2,1,H,True,False
4,5,D2,2009,2009-12-06,Ahlen,Karlsruhe,1,3,A,False,False


In [72]:
df_matches['winner2'] = df_matches.map(lambda x: if x in df_matches['FTR'] == 'H', axis=1)

SyntaxError: invalid syntax (<ipython-input-72-2620120bc0c0>, line 1)

In [54]:
cur.execute("""SELECT * FROM unique_teams
            JOIN teams_in_matches
            USING (unique_team_id)
            JOIN matches
            USING (match_id)
            WHERE season == 2011
            
            """)

df_3_tables = pd.DataFrame(cur.fetchall())
df_3_tables.columns = [x[0] for x in cur.description]
df_3_tables.head()

,TeamName,Unique_Team_ID,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,Bayern Munich,1,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,Nurnberg,22,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
2,Bayern Munich,1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
3,Stuttgart,17,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
4,Bayern Munich,1,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A


In [64]:
for i, row in df_3_tables.iterrows():
    if row.TeamName == row.HomeTeam:
        if row.FTHG > row.FTAG:
            df_3_tables.loc[i,'W'] = "1"
        elif row.FTHG < row.FTAG:
            df_3_tables.loc[i,'L'] = "1"
        elif row.FTHG == row.FTAG:
            df_3_tables.loc[i,'D'] = "1"
    else:
        if row.FTAG > row.FTHG:
            df_3_tables.loc[i,'W'] = "1"
        elif row.FTAG < row.FTHG:
            df_3_tables.loc[i,'L'] = "1"
        elif row.FTAG == row.FTHG:
            df_3_tables.loc[i,'D'] = "1"

In [65]:
df_3_tables.head()

,TeamName,Unique_Team_ID,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,W,L,D
0,Bayern Munich,1,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,1,NaN,NaN
1,Nurnberg,22,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,NaN,1,NaN
2,Bayern Munich,1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,1,NaN,NaN
3,Stuttgart,17,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,NaN,1,NaN
4,Bayern Munich,1,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,1,NaN,NaN


In [ ]:
df_matches.head()

In [ ]:
for i in (high_scores1):
  if(high_scores1['runs1'][i]>=high_scores1['runs2'][i]):
      high_scores1['is_score_chased'][i]='yes'
  else:
      high_scores1['is_score_chased'][i]='no' 

In [58]:
for i in (df_matches):
  if(df_matches['FTHG'][i]>=df_matches['FTR'][i]):
      df_matches['winner2'][i]='yes'
  else:
      df_matches['winner2'][i]='no' 

KeyError: 'Match_ID'

In [56]:
for x in (df_matches):
    if (df_matches['FTR'][x] == "H"):
        df_matches['winner2'][x] = df_matches['HomeTeam'][x]
    else:
        df_matches['winner2'][x] = df_matches['AwayTeam'][x]

KeyError: 'Match_ID'

In [49]:
df_matches['winner2'] = df_matches(df_matches[''])

In [52]:
df_matches['winner2'] = df_matches['HomeTeam'].map(lambda x: )

In [53]:
df_matches.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,winner,winner2
0,1,D2,2009,2010-04-04,Oberhausen,Kaiserslautern,2,1,H,True,10
1,2,D2,2009,2009-11-01,Munich 1860,Kaiserslautern,0,1,A,False,11
2,3,D2,2009,2009-10-04,Frankfurt FSV,Kaiserslautern,1,1,D,False,13
3,4,D2,2009,2010-02-21,Frankfurt FSV,Karlsruhe,2,1,H,True,13
4,5,D2,2009,2009-12-06,Ahlen,Karlsruhe,1,3,A,False,5


In [7]:
cur.execute("""SELECT * FROM teams""")

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,Season,TeamName,KaderHome,AvgAgeHome,ForeignPlayersHome,OverallMarketValueHome,AvgMarketValueHome,StadiumCapacity
0,2017,Bayern Munich,27,26,15,597950000,22150000,75000
1,2017,Dortmund,33,25,18,416730000,12630000,81359
2,2017,Leverkusen,31,24,15,222600000,7180000,30210
3,2017,RB Leipzig,30,23,15,180130000,6000000,42959
4,2017,Schalke 04,29,24,17,179550000,6190000,62271


In [8]:
cur.execute("""SELECT * FROM teams_in_matches""")

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,Match_ID,Unique_Team_ID
0,1,26
1,1,46
2,2,26
3,2,42
4,3,26


In [9]:
cur.execute("""SELECT * FROM unique_teams""")

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,TeamName,Unique_Team_ID
0,Bayern Munich,1
1,Dortmund,2
2,Leverkusen,3
3,RB Leipzig,4
4,Schalke 04,5


In [17]:
cur.execute("""SELECT * FROM unique_teams
            JOIN teams_in_matches
            USING (unique_team_id)
            ORDER BY unique_team_id ASC""")

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,TeamName,Unique_Team_ID,Match_ID
0,Bayern Munich,1,22
1,Bayern Munich,1,49
2,Bayern Munich,1,57
3,Bayern Munich,1,78
4,Bayern Munich,1,100


In [44]:
cur.execute("""SELECT * FROM unique_teams
            JOIN teams_in_matches
            USING (unique_team_id)
            JOIN matches
            USING (match_id)
            WHERE season == 2011
            GROUP BY match_id
            """)

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,TeamName,Unique_Team_ID,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,Bayern Munich,1,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,Bayern Munich,1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
2,Bayern Munich,1,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
3,Bayern Munich,1,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
4,Bayern Munich,1,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D


In [112]:
cur.execute("""SELECT * FROM unique_teams
            JOIN teams_in_matches
            USING (unique_team_id)
            JOIN matches
            USING (match_id)
            WHERE season == 2011
                        
            """)

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,TeamName,Unique_Team_ID,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,Bayern Munich,1,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,Nurnberg,22,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
2,Bayern Munich,1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
3,Stuttgart,17,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
4,Bayern Munich,1,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A


In [ ]:
def test_iterrow(df):
    for (i, row) in df.iterrows():
        val = row['Age']
        if math.isnan(val):
                row['Age'] = -1
%timeit test_iterrow(titanic_df)

In [362]:
df.head()

,TeamName,Unique_Team_ID,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Outcome,Total_Goals_by_Team_Name,Date Unix
0,Bayern Munich,1,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,W,1.0,1333152000
1,Nurnberg,22,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,L,0.0,1333152000
2,Bayern Munich,1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,W,2.0,1323561600
3,Stuttgart,17,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,L,1.0,1323561600
4,Bayern Munich,1,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,W,1.0,1313193600


In [82]:
import numpy as np

In [90]:
np.max(df['Date'])

'2012-05-13'

In [91]:
np.min(df['Date'])

'2011-07-15'

In [ ]:
cur.execute("""SELECT TeamName,  FROM unique_teams
            JOIN teams_in_matches
            USING (unique_team_id)
            JOIN matches
            USING (match_id)
            WHERE season == 2011
            
            """)

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

In [104]:
len(df['Match_ID'].unique())

992

In [9]:
import json
import pandas as pd
import requests


In [11]:
# checking the status code, response we got here is 200, means that it was able to connect
# to the API successfully
print(response.status_code)

200


In [400]:
print(type(response.text))

<class 'str'>


In [12]:
#checking the response text
print(response.text)

{"latitude":42.3601,"longitude":-71.0589,"timezone":"America/New_York","currently":{"time":255657600,"summary":"Snow and Dangerously Windy","icon":"snow","precipIntensity":0.1692,"precipProbability":1,"precipType":"snow","precipAccumulation":1.7634,"temperature":30.89,"apparentTemperature":14.34,"dewPoint":30.8,"humidity":1,"pressure":1005.7,"windSpeed":39.46,"windGust":96.13,"windBearing":64,"cloudCover":1,"uvIndex":0,"visibility":0.201},"hourly":{"summary":"Snow throughout the day.","icon":"snow","data":[{"time":255589200,"summary":"Overcast","icon":"cloudy","precipIntensity":0,"precipProbability":0,"temperature":23.94,"apparentTemperature":18.11,"dewPoint":16.98,"humidity":0.74,"pressure":1026.8,"windSpeed":4.55,"windBearing":350,"cloudCover":0.88,"uvIndex":0,"visibility":9.997},{"time":255592800,"summary":"Mostly Cloudy","icon":"partly-cloudy-night","precipIntensity":0,"precipProbability":0,"temperature":24.08,"apparentTemperature":14.45,"dewPoint":17.74,"humidity":0.76,"pressure":

In [401]:
response.json().keys()

dict_keys(['code', 'error'])

In [13]:
# creating a data object to apply json function to the response
data = response.json()

In [399]:
print(data.keys())

Index(['latitude', 'longitude', 'timezone', 'currently', 'hourly', 'daily',
       'flags', 'offset'],
      dtype='object')


In [14]:
# putting the data object into a pandas dataframe
data = pd.DataFrame(data)

In [368]:
# checking the head to see what the dataframe looks like
data.head()

,latitude,longitude,timezone,currently,hourly,daily,flags,offset
apparentTemperature,42.3601,-71.0589,America/New_York,14.34,NaN,NaN,NaN,-5
cloudCover,42.3601,-71.0589,America/New_York,1,NaN,NaN,NaN,-5
data,42.3601,-71.0589,America/New_York,NaN,"[{'time': 255589200, 'summary': 'Overcast', 'i...","[{'time': 255589200, 'summary': 'Snow and dang...",NaN,-5
dewPoint,42.3601,-71.0589,America/New_York,30.8,NaN,NaN,NaN,-5
humidity,42.3601,-71.0589,America/New_York,1,NaN,NaN,NaN,-5


In [398]:
keys = data.daily.iloc[2]
keys


[{'time': 255589200,
  'summary': 'Snow and dangerously windy throughout the day.',
  'icon': 'snow',
  'sunriseTime': 255614040,
  'sunsetTime': 255650760,
  'moonPhase': 0.98,
  'precipIntensity': 0.0328,
  'precipIntensityMax': 0.1745,
  'precipIntensityMaxTime': 255658620,
  'precipProbability': 1,
  'precipType': 'snow',
  'precipAccumulation': 5.96,
  'temperatureHigh': 34.59,
  'temperatureHighTime': 255630540,
  'temperatureLow': 29.51,
  'temperatureLowTime': 255697260,
  'apparentTemperatureHigh': 23.81,
  'apparentTemperatureHighTime': 255629340,
  'apparentTemperatureLow': 12.5,
  'apparentTemperatureLowTime': 255668280,
  'dewPoint': 26.38,
  'humidity': 0.87,
  'pressure': 1016.6,
  'windSpeed': 23.39,
  'windGust': 97.39,
  'windGustTime': 255658260,
  'windBearing': 57,
  'cloudCover': 0.95,
  'uvIndex': 2,
  'uvIndexTime': 255632520,
  'visibility': 5.502,
  'temperatureMin': 23.32,
  'temperatureMinTime': 255598740,
  'temperatureMax': 34.59,
  'temperatureMaxTime': 2

In [277]:
data.iloc[3:]['daily']

dewPoint              NaN
humidity              NaN
icon                  NaN
nearest-station       NaN
precipAccumulation    NaN
precipIntensity       NaN
precipProbability     NaN
precipType            NaN
pressure              NaN
sources               NaN
summary               NaN
temperature           NaN
time                  NaN
units                 NaN
uvIndex               NaN
visibility            NaN
windBearing           NaN
windGust              NaN
windSpeed             NaN
Name: daily, dtype: object

In [366]:
def get_keys(path):
    with open(path) as f:
        return json.load(f)

In [367]:
keys = get_keys("/Users/naweedahmed/.secret/darksky_api.json")

api_key = keys['api_key']

In [376]:


import requests
latitude = 52.52
longitude = 13.4050
time = 255657600
icon = 'rain'
SEARCH_LIMIT = 10


url = 'https://api.darksky.net/forecast/xx'


url_params = {
                'latitude': latitude,
                'longitude': longitude,
                'icon': icon,
                'limit': SEARCH_LIMIT
            }

response = requests.get(url, params=url_params)
print(response)
print(type(response.text))
print(response.text[:1000])



<Response [400]>
<class 'str'>
{"code":400,"error":"Poorly formatted request"}

